# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-8f7f46850d-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Movie Workflow Series ("Film Genre and directors explorative search") 

Consider the following exploratory information need:

> investigate the results concerning the different film genre over years and the directors for the cinema.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | occupation    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wdt:P3342`     | Significant person       | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q25089`| Woody Allen       | node |
| `wd:Q3772`| Quentin Tarantino      | node |






Also consider

```
wd:Q25089 ?p ?obj .
```

is the BGP to retrieve all **properties of Woody Allen**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.

The workload should:


1. Identify the BGP for films

2. Identify the BGP for director

3. Identify the BGP for film genre

4. Find how many western films are been released in each european country from 2010-01-01 to 2015-31-12 (the result set must be country IRI, label and #films).

5. Consider only films directed by Woody Allen or Quentin Tarantino. Then consider the date of publication of these films (take the first one if there are more than one date of publication). Return the year with more films release for the two director (the result set must be two elements and for each element there must be present the director IRI, the label of the IRI, the year and the number of films). 

6. Identify the BGP for Academy Awards 

7. Consider films directed by Woody Allen or Quentin Tarantino separately. Return the film which won more Academy Awards for the two directors (the result set must be two elements - one related to Allen and the other related to Tarantino -  and for each element there must be present the director IRI with its label, the film IRI with its label and the number of Academy awards won).

8. Consider the set of films directed by Allen or Tarantino. Return for each Academy Award the total number of nomination of these films (the result set must be a list of triples Academy Award IRI, label and #nomination).

## Task 1
Identify the BGP for films

In [3]:
# query example

#Query 1

# At first let check what is/are Woody Allen's ocupation.
# wd:Q25089 => Woody Allen
# wdt:P106  => occupation

queryString = """
SELECT DISTINCT ?o ?oName
WHERE {
   # bind something
   wd:Q25089 wdt:P106 ?o .
   
   # get the label
   ?o sc:name ?oName.
}

LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q10800557'), ('oName', 'film actor')]
[('o', 'http://www.wikidata.org/entity/Q15981151'), ('oName', 'jazz musician')]
[('o', 'http://www.wikidata.org/entity/Q18844224'), ('oName', 'science fiction writer')]
[('o', 'http://www.wikidata.org/entity/Q1930187'), ('oName', 'journalist')]
[('o', 'http://www.wikidata.org/entity/Q214917'), ('oName', 'playwright')]
[('o', 'http://www.wikidata.org/entity/Q245068'), ('oName', 'comedian')]
[('o', 'http://www.wikidata.org/entity/Q2526255'), ('oName', 'film director')]
[('o', 'http://www.wikidata.org/entity/Q28389'), ('oName', 'screenwriter')]
[('o', 'http://www.wikidata.org/entity/Q3282637'), ('oName', 'film producer')]
[('o', 'http://www.wikidata.org/entity/Q36180'), ('oName', 'writer')]
[('o', 'http://www.wikidata.org/entity/Q36834'), ('oName', 'composer')]
[('o', 'http://www.wikidata.org/entity/Q482980'), ('oName', 'author')]
[('o', 'http://www.wikidata.org/entity/Q639669'), ('oName', 'musician')]
[(

In [4]:
#Query 2

# He could have different roles related to films. I am looking for object which is instance of film. I can filter them for things like 'film' or 'movie',
# but I decided no to do it. Because maybe in the results I find somethings helpfull for other tasks.
# wd:Q25089 => Woody Allen
# wdt:P106  => occupation
# wdt:P31 => instance of

queryString = """
SELECT DISTINCT ?o ?oName ?m ?mName
WHERE {
   # bind something
   wd:Q25089 ?p ?o .
   ?o wdt:P31 ?m.
      
   # get the label
   ?m sc:name ?mName.
   ?o sc:name ?oName.
   
}
ORDER BY (?mName)
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q103360'), ('oName', 'Academy Award for Best Director'), ('m', 'http://www.wikidata.org/entity/Q19020'), ('mName', 'Academy Awards')]
[('o', 'http://www.wikidata.org/entity/Q41417'), ('oName', 'Academy Award for Best Writing, Original Screenplay'), ('m', 'http://www.wikidata.org/entity/Q19020'), ('mName', 'Academy Awards')]
[('o', 'http://www.wikidata.org/entity/Q103916'), ('oName', 'Academy Award for Best Actor'), ('m', 'http://www.wikidata.org/entity/Q19020'), ('mName', 'Academy Awards')]
[('o', 'http://www.wikidata.org/entity/Q139184'), ('oName', 'BAFTA Award for Best Film'), ('m', 'http://www.wikidata.org/entity/Q732997'), ('mName', 'British Academy Film Awards')]
[('o', 'http://www.wikidata.org/entity/Q787148'), ('oName', 'Academy Fellowship Award'), ('m', 'http://www.wikidata.org/entity/Q732997'), ('mName', 'British Academy Film Awards')]
[('o', 'http://www.wikidata.org/entity/Q787131'), ('oName', 'BAFTA Award for Best Direction'), (

In [5]:
#Query 3

# wd:Q11424 => film

queryString = """
SELECT DISTINCT (wd:Q11424 as ?IRI) ?name
WHERE {
   # bind something
   ?s wdt:P31 wd:Q11424 .
   
   # get the label
   wd:Q11424 sc:name ?name.
}

LIMIT 1
"""

print("Results")
x=run_query(queryString)

Results
[('IRI', 'http://www.wikidata.org/entity/Q11424'), ('name', 'film')]
1


Final query for this task

In [6]:
# write your final query

#Query 4


# wd:Q11424 => film

queryString = """
SELECT DISTINCT (wd:Q11424 as ?IRI) ?name
WHERE {
   # bind something
   ?s wdt:P31 wd:Q11424 .
   
   # get the label
   wd:Q11424 sc:name ?name.
}

LIMIT 1
"""

print("Results")
x=run_query(queryString)

Results
[('IRI', 'http://www.wikidata.org/entity/Q11424'), ('name', 'film')]
1


## Task 2
Identify the BGP for director

In [7]:
# write your queries

# Query 5

# One of Woody Allen's occupation is film director. let check its superclass.
# wd:Q2526255 => film director (Query 1)

queryString = """
SELECT DISTINCT ?o ?oName
WHERE {
   # bind something
   wd:Q2526255 wdt:P279 ?o  .
   
   # get the label
   ?o  sc:name ?oName.
}
ORDER BY ?pName
LIMIT 200
"""

print("Results")
x=run_query(queryString)


Results
[('o', 'http://www.wikidata.org/entity/Q3455803'), ('oName', 'director')]
[('o', 'http://www.wikidata.org/entity/Q3391743'), ('oName', 'visual artist')]
[('o', 'http://www.wikidata.org/entity/Q3406858'), ('oName', 'film professional')]
3


Final query for this task

In [8]:
# write your final query

#Query 6

# wd:Q3455803 => director    (Query 5)

queryString = """
SELECT DISTINCT (wd:Q3455803 as ?IRI) ?name
WHERE {
   # bind something
   
   # get the label
   wd:Q3455803 sc:name ?name.
}

LIMIT 1
"""

print("Results")
x=run_query(queryString)

Results
[('IRI', 'http://www.wikidata.org/entity/Q3455803'), ('name', 'director')]
1


## Task 3
Identify the BGP for film genre

In [9]:
# write your queries

#Query 7

# Ckecking all properties of a film related to Woody Allen to find 'genre'
# wd:Q206124 => 'Midnight in Paris'  (Query 2)

queryString = """
SELECT DISTINCT ?p ?pName
WHERE {
   # bind something
   wd:Q206124 ?p ?o  .
   
   # get the label
   ?p  sc:name ?pName.
}
ORDER BY ?pName
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P3593'), ('pName', 'AFI Catalog of Feature Films ID')]
[('p', 'http://www.wikidata.org/prop/direct/P6145'), ('pName', 'Academy Awards Database film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1562'), ('pName', 'AllMovie title ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2465'), ('pName', 'Allcinema film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1265'), ('pName', 'AlloCiné film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P9586'), ('pName', 'Apple TV movie ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2629'), ('pName', 'BBFC rating')]
[('p', 'http://www.wikidata.org/prop/direct/P4438'), ('pName', 'BFI Films, TV and people ID')]
[('p', 'http://www.wikidata.org/prop/direct/P4632'), ('pName', 'Bechdel Test Movie List ID')]
[('p', 'http://www.wikidata.org/prop/direct/P950'), ('pName', 'Biblioteca Nacional de España ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1237'), ('pName', 'Box Office Mojo film ID 

In [10]:
# Query 8

# Finding classes that genre of 'Midnight in Paris' is an instance of it.

# wd:Q206124 => 'Midnight in Paris'  (Query 2)
# wdt:P136 => genre    (Query 7)

queryString = """
SELECT DISTINCT ?g ?gName
WHERE {
   # bind something
   wd:Q206124 wdt:P136 ?o .
   ?o wdt:P31 ?g.
   
   # get the label
   ?g sc:name ?gName.
}

LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('g', 'http://www.wikidata.org/entity/Q108329096'), ('gName', 'genre by art form')]
[('g', 'http://www.wikidata.org/entity/Q15961987'), ('gName', 'television genre')]
[('g', 'http://www.wikidata.org/entity/Q201658'), ('gName', 'film genre')]
[('g', 'http://www.wikidata.org/entity/Q1194240'), ('gName', 'science fiction genre')]
[('g', 'http://www.wikidata.org/entity/Q20076756'), ('gName', 'speculative fiction genre')]
[('g', 'http://www.wikidata.org/entity/Q5151404'), ('gName', 'comedic genre')]
6


Final query for this task

In [11]:
# write your final query


#Query 9

# wd:Q201658 => film genre

queryString = """
SELECT DISTINCT (wd:Q201658 as ?IRI) ?name
WHERE {
   # bind something

   # get the label
   wd:Q201658 sc:name ?name.
}

LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('IRI', 'http://www.wikidata.org/entity/Q201658'), ('name', 'film genre')]
1


## Task 4
Find how many western films are been released in each european country from 2010-01-01 to 2015-31-12 (the result set must be country IRI, label and #films).

In [12]:
# write your queries

# Query 10

# To complete this task, I need to find 'western', 'realeased date', 'european',and 'country' (or similar/sysnonymum things).
# From another workflow I know that
    # wdt:P17 => 'country'
    # wdt:P30 => 'continent'
    # wd:Q46 => 'Europe'
# However, I suppose every workflow as totally independent work. Thus I will try to find them again.

# Usefull thing that I have found before are:
# wdt:P577 => publication date  (Query 7)  ----> instead of released date
# wdt:P495 => country of origin (Query 7)
# wdt:P136 => genre    (Query 7)

# First let find western.

queryString = """
SELECT DISTINCT ?g ?gName
WHERE {
   # bind something
   ?g wdt:P31 wd:Q201658.
   
   FILTER(REGEX(?gName, "[Ww]estern"))
   
   # get the label
   ?g sc:name ?gName.
}
ORDER BY ?oName
LIMIT 100
"""

print("Results")
x=run_query(queryString)



Results
[('g', 'http://www.wikidata.org/entity/Q268854'), ('gName', 'Red Western')]
[('g', 'http://www.wikidata.org/entity/Q4235011'), ('gName', 'Space Western')]
[('g', 'http://www.wikidata.org/entity/Q172980'), ('gName', 'Western film')]
[('g', 'http://www.wikidata.org/entity/Q491158'), ('gName', 'Revisionist Western')]
[('g', 'http://www.wikidata.org/entity/Q212781'), ('gName', 'spaghetti western')]
[('g', 'http://www.wikidata.org/entity/Q4836991'), ('gName', 'B western')]
[('g', 'http://www.wikidata.org/entity/Q6804274'), ('gName', 'meat pie Western')]
[('g', 'http://www.wikidata.org/entity/Q5461696'), ('gName', 'Florida Western')]
[('g', 'http://www.wikidata.org/entity/Q4674071'), ('gName', 'Acid Western')]
[('g', 'http://www.wikidata.org/entity/Q83646243'), ('gName', 'western animation')]
[('g', 'http://www.wikidata.org/entity/Q78528852'), ('gName', 'neo-western')]
[('g', 'http://www.wikidata.org/entity/Q8066387'), ('gName', 'Zapata Western')]
12


In [13]:

# Query 11

# Finding sample country will help to find a property related to 'european'

# wdt:P577 => publication date 
# wdt:P495 => country of origin
# wdt:P136 => genre
# wd:Q172980 => Western film (Query 10)


queryString = """
SELECT DISTINCT ?o ?oName
WHERE {
   # bind something
   ?s wdt:P495 ?o.
      
   # get the label
   ?o sc:name ?oName.
}
ORDER BY ?oName
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q12536'), ('oName', 'Abbasid Caliphate')]
[('o', 'http://www.wikidata.org/entity/Q23334'), ('oName', 'Abkhazia')]
[('o', 'http://www.wikidata.org/entity/Q1519'), ('oName', 'Abu Dhabi')]
[('o', 'http://www.wikidata.org/entity/Q207353'), ('oName', 'Acadia')]
[('o', 'http://www.wikidata.org/entity/Q1823'), ('oName', 'Aceh')]
[('o', 'http://www.wikidata.org/entity/Q389688'), ('oName', 'Achaemenid Empire')]
[('o', 'http://www.wikidata.org/entity/Q131013'), ('oName', 'Acropolis of Athens')]
[('o', 'http://www.wikidata.org/entity/Q666909'), ('oName', 'Act of Mediation')]
[('o', 'http://www.wikidata.org/entity/Q5112'), ('oName', 'Adelaide')]
[('o', 'http://www.wikidata.org/entity/Q131694'), ('oName', 'Aden')]
[('o', 'http://www.wikidata.org/entity/Q1152445'), ('oName', 'Aerican Empire')]
[('o', 'http://www.wikidata.org/entity/Q889'), ('oName', 'Afghanistan')]
[('o', 'http://www.wikidata.org/entity/Q15'), ('oName', 'Africa')]
[('o', 'http://www.wik

In [14]:

# Query 12

# Results of Query 11 not only includes countries, but also continet and cities. 
# let use it with filtering to reach 'country' and 'european' more quickly.

# Usefull thing that I have found before are:
# wdt:P577 => publication date 
# wdt:P495 => country of origin
# wdt:P136 => genre
# wd:Q172980 => Western film

# wd:Q222 => Albania (Query 11) ----> an european country

queryString = """
SELECT DISTINCT ?o ?oName
WHERE {
   # bind something
   ?s wdt:P495 ?o.
   
   FILTER(REGEX(?oName, "[Ee]urop"))
   
   # get the label
   ?o sc:name ?oName.
}
ORDER BY ?oName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q27509'), ('oName', 'Central Europe')]
[('o', 'http://www.wikidata.org/entity/Q27468'), ('oName', 'Eastern Europe')]
[('o', 'http://www.wikidata.org/entity/Q46'), ('oName', 'Europe')]
[('o', 'http://www.wikidata.org/entity/Q166400'), ('oName', 'European Broadcasting Union')]
[('o', 'http://www.wikidata.org/entity/Q458'), ('oName', 'European Union')]
[('o', 'http://www.wikidata.org/entity/Q27479'), ('oName', 'Northern Europe')]
[('o', 'http://www.wikidata.org/entity/Q27449'), ('oName', 'Southern Europe')]
[('o', 'http://www.wikidata.org/entity/Q27496'), ('oName', 'Western Europe')]
[('o', 'http://www.wikidata.org/entity/Q466039'), ('oName', 'continental Europe')]
9


In [15]:

# Query 13

# Finding relation between 'Albania' and 'Europe'

# Usefull thing that I have found before are:
# wdt:P577 => publication date 
# wdt:P495 => country of origin
# wdt:P136 => genre
# wd:Q172980 => Western film
# wd:Q46 => Europe

# wd:Q222 => Albania (Query 11) ----> an european country

queryString = """
SELECT DISTINCT ?p ?pName
WHERE {
   # bind something
   wd:Q222 ?p wd:Q46.
   
   
   # get the label
   ?p sc:name ?pName.
}
ORDER BY ?oName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P30'), ('pName', 'continent')]
1


In [39]:

# Query 14

# TASK 4:
    # Find how many western films are been released in each european country from 2010-01-01 to 2015-31-12 
    # (the result set must be country IRI, label and #films).
    
# Let find all western films from european countries from 2010-01-01 to 2015-31-12 to check if everythings is correct.

# Usefull thing that I have found before are:
# wdt:P577 => publication date 
# wdt:P495 => country of origin
# wdt:P136 => genre
# wd:Q172980 => Western film
# wd:Q46 => Europe
# wdt:P30 => continent
# wd:Q11424 => film


queryString = """
SELECT DISTINCT ?c as ?IRI ?Country  ?Film ?d as ?Releasedate
WHERE {
   # bind something
   ?f wdt:P31 wd:Q11424 ;
      wdt:P136 wd:Q172980 ;
      wdt:P495 ?c ;
      wdt:P577 ?d .
      
   ?c wdt:P30 wd:Q46 .

   FILTER(2010 <= year(?d) && year(?d) <= 2015 )
   
   # get the label
   ?c sc:name ?Country.
   ?f sc:name ?Film.
}
ORDER BY ?Country
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('IRI', 'http://www.wikidata.org/entity/Q40'), ('Country', 'Austria'), ('Film', 'The Dark Valley'), ('Releasedate', '2014-02-10T00:00:00Z')]
[('IRI', 'http://www.wikidata.org/entity/Q40'), ('Country', 'Austria'), ('Film', 'The Dark Valley'), ('Releasedate', '2014-02-13T00:00:00Z')]
[('IRI', 'http://www.wikidata.org/entity/Q40'), ('Country', 'Austria'), ('Film', 'The Dark Valley'), ('Releasedate', '2014-02-14T00:00:00Z')]
[('IRI', 'http://www.wikidata.org/entity/Q31'), ('Country', 'Belgium'), ('Film', 'The Salvation'), ('Releasedate', '2014-01-01T00:00:00Z')]
[('IRI', 'http://www.wikidata.org/entity/Q31'), ('Country', 'Belgium'), ('Film', 'The Salvation'), ('Releasedate', '2014-10-09T00:00:00Z')]
[('IRI', 'http://www.wikidata.org/entity/Q35'), ('Country', 'Denmark'), ('Film', 'The Salvation'), ('Releasedate', '2014-01-01T00:00:00Z')]
[('IRI', 'http://www.wikidata.org/entity/Q35'), ('Country', 'Denmark'), ('Film', 'The Salvation'), ('Releasedate', '2014-10-09T00:00:00Z')]
[('IRI

Final query for this task

In [3]:
# write your final query

# Query 15

# TASK 4:
    # Find how many western films are been released in each european country from 2010-01-01 to 2015-31-12 
    # (the result set must be country IRI, label and #films).
    
# Query 14 shows that some films has more than one release date. They should be count once.

# Usefull thing that I have found before are:
# wdt:P577 => publication date 
# wdt:P495 => country of origin
# wdt:P136 => genre
# wd:Q172980 => Western film
# wd:Q46 => Europe
# wdt:P30 => continent
# wd:Q11424 => film


queryString = """
SELECT DISTINCT ?c as ?IRI ?Country  (COUNT(DISTINCT(?f)) as ?noOfFilms)
WHERE {
   # bind something
   ?f wdt:P31 wd:Q11424 ;
      wdt:P136 wd:Q172980 ;
      wdt:P495 ?c ;
      wdt:P577 ?d .
      
   ?c wdt:P30 wd:Q46 .
   
   FILTER(2010 <= year(?d) && year(?d) <= 2015 )
   
   # get the label
   ?c sc:name ?Country.
   ?f sc:name ?Film.
}
ORDER BY ASC(?Country)
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('IRI', 'http://www.wikidata.org/entity/Q40'), ('Country', 'Austria'), ('noOfFilms', '1')]
[('IRI', 'http://www.wikidata.org/entity/Q31'), ('Country', 'Belgium'), ('noOfFilms', '1')]
[('IRI', 'http://www.wikidata.org/entity/Q35'), ('Country', 'Denmark'), ('noOfFilms', '1')]
[('IRI', 'http://www.wikidata.org/entity/Q142'), ('Country', 'France'), ('noOfFilms', '2')]
[('IRI', 'http://www.wikidata.org/entity/Q183'), ('Country', 'Germany'), ('noOfFilms', '3')]
[('IRI', 'http://www.wikidata.org/entity/Q221'), ('Country', 'North Macedonia'), ('noOfFilms', '1')]
[('IRI', 'http://www.wikidata.org/entity/Q29'), ('Country', 'Spain'), ('noOfFilms', '1')]
[('IRI', 'http://www.wikidata.org/entity/Q34'), ('Country', 'Sweden'), ('noOfFilms', '1')]
[('IRI', 'http://www.wikidata.org/entity/Q43'), ('Country', 'Turkey'), ('noOfFilms', '1')]
[('IRI', 'http://www.wikidata.org/entity/Q145'), ('Country', 'United Kingdom'), ('noOfFilms', '3')]
10


## Task 5
Consider only films directed by Woody Allen or Quentin Tarantino. Then consider the date of publication of these films (take the first one if there are more than one date of publication). Return the year with more films release for the two director (the result set must be two elements and for each element there must be present the director IRI, the label of the IRI, the year and the number of films).

In [9]:
# write your queries

# Query 16

# Finding films directed by Woody Allen and their details to have a clue.
# Sorting the result is just for make it easier to review.

# wd:Q25089 => Woody Allen
# wd:Q3772 => Quentin Tarantino
# wdt:P57 => director  (Query 7)
# wdt:P577 => publication date

queryString = """
SELECT DISTINCT ?film ?pdate
WHERE {
   # bind something
  ?f wdt:P31 wd:Q11424 ;
     wdt:P57 wd:Q25089 ;
     wdt:P577 ?pdate .
   
   
   # get the label
   ?f sc:name ?film.

}
ORDER BY (?film)  (?pdate) 
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('film', "A Midsummer Night's Sex Comedy"), ('pdate', '1982-01-01T00:00:00Z')]
[('film', "A Midsummer Night's Sex Comedy"), ('pdate', '1982-10-15T00:00:00Z')]
[('film', 'A Rainy Day in New York'), ('pdate', '2019-07-26T00:00:00Z')]
[('film', 'A Rainy Day in New York'), ('pdate', '2019-09-18T00:00:00Z')]
[('film', 'A Rainy Day in New York'), ('pdate', '2019-12-05T00:00:00Z')]
[('film', 'A Rainy Day in New York'), ('pdate', '2019-12-12T00:00:00Z')]
[('film', 'Alice'), ('pdate', '1990-01-01T00:00:00Z')]
[('film', 'Alice'), ('pdate', '1991-02-28T00:00:00Z')]
[('film', 'Annie Hall'), ('pdate', '1977-04-20T00:00:00Z')]
[('film', 'Another Woman'), ('pdate', '1988-01-01T00:00:00Z')]
[('film', 'Another Woman'), ('pdate', '1989-03-16T00:00:00Z')]
[('film', 'Anything Else'), ('pdate', '2003-01-01T00:00:00Z')]
[('film', 'Bananas'), ('pdate', '1971-01-01T00:00:00Z')]
[('film', 'Bananas'), ('pdate', '1974-08-30T00:00:00Z')]
[('film', 'Blue Jasmine'), ('pdate', '2013-07-26T00:00:00Z')]
[('fi

In [29]:

# Query 17

# Lots of films have more than one publication date.
# Selecting first publication date for each film.

# Sorting the result is just for make it easier to review.

# wd:Q25089 => Woody Allen
# wd:Q3772 => Quentin Tarantino
# wdt:P57 => director  (Query 7)
# wdt:P577 => publication date

queryString = """
SELECT DISTINCT ?film (MIN(YEAR(?pdate)) as ?pYear)
WHERE {
   # bind something
  ?f wdt:P31 wd:Q11424 ;
     wdt:P57 wd:Q25089 ;
     wdt:P577 ?pdate .
   
   # get the label
   ?f sc:name ?film.

}
ORDER BY ASC(?pYear) (?film)
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('film', "What's Up, Tiger Lily?"), ('pYear', '1966')]
[('film', 'Take the Money and Run'), ('pYear', '1969')]
[('film', 'Bananas'), ('pYear', '1971')]
[('film', 'Everything You Always Wanted to Know About Sex*'), ('pYear', '1972')]
[('film', 'Sleeper'), ('pYear', '1973')]
[('film', 'Love and Death'), ('pYear', '1975')]
[('film', 'Annie Hall'), ('pYear', '1977')]
[('film', 'Interiors'), ('pYear', '1978')]
[('film', 'Manhattan'), ('pYear', '1979')]
[('film', 'Stardust Memories'), ('pYear', '1980')]
[('film', "A Midsummer Night's Sex Comedy"), ('pYear', '1982')]
[('film', 'Zelig'), ('pYear', '1983')]
[('film', 'Broadway Danny Rose'), ('pYear', '1984')]
[('film', 'The Purple Rose of Cairo'), ('pYear', '1985')]
[('film', 'Hannah and Her Sisters'), ('pYear', '1986')]
[('film', 'Radio Days'), ('pYear', '1987')]
[('film', 'September'), ('pYear', '1987')]
[('film', 'Another Woman'), ('pYear', '1988')]
[('film', 'Crimes and Misdemeanors'), ('pYear', '1989')]
[('film', 'New York Stories

Final query for this task

In [11]:
# Query 18

# Counting no of films for each year.
# Sorting the result is just for make it easier to review.

# wd:Q25089 => Woody Allen
# wd:Q3772 => Quentin Tarantino
# wdt:P57 => director  (Query 7)
# wdt:P577 => publication date

queryString = """
SELECT DISTINCT ?pYear (COUNT(?pYear) as ?filmNo) WHERE
{
    SELECT DISTINCT ?film (MIN(YEAR(?pdate)) as ?pYear) WHERE
    {
        # bind something
        ?f wdt:P31 wd:Q11424 ;
            wdt:P57 wd:Q25089 ;
            wdt:P577 ?pdate .
        
        # get the label
        ?f sc:name ?film.
    }
    GROUP BY ?film
}
GROUP BY ?pYear
ORDER BY ?pYear
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('pYear', '1966'), ('filmNo', '1')]
[('pYear', '1969'), ('filmNo', '1')]
[('pYear', '1971'), ('filmNo', '1')]
[('pYear', '1972'), ('filmNo', '1')]
[('pYear', '1973'), ('filmNo', '1')]
[('pYear', '1975'), ('filmNo', '1')]
[('pYear', '1977'), ('filmNo', '1')]
[('pYear', '1978'), ('filmNo', '1')]
[('pYear', '1979'), ('filmNo', '1')]
[('pYear', '1980'), ('filmNo', '1')]
[('pYear', '1982'), ('filmNo', '1')]
[('pYear', '1983'), ('filmNo', '1')]
[('pYear', '1984'), ('filmNo', '1')]
[('pYear', '1985'), ('filmNo', '1')]
[('pYear', '1986'), ('filmNo', '1')]
[('pYear', '1987'), ('filmNo', '2')]
[('pYear', '1988'), ('filmNo', '1')]
[('pYear', '1989'), ('filmNo', '2')]
[('pYear', '1990'), ('filmNo', '1')]
[('pYear', '1991'), ('filmNo', '1')]
[('pYear', '1992'), ('filmNo', '1')]
[('pYear', '1993'), ('filmNo', '1')]
[('pYear', '1994'), ('filmNo', '1')]
[('pYear', '1995'), ('filmNo', '1')]
[('pYear', '1996'), ('filmNo', '1')]
[('pYear', '1997'), ('filmNo', '1')]
[('pYear', '1998'), ('filmNo',

In [13]:
# Query 19

# Adding both directors to query. 
# Sorting the result is just for make it easier to review.

# wd:Q25089 => Woody Allen
# wd:Q3772 => Quentin Tarantino
# wdt:P57 => director  (Query 7)
# wdt:P577 => publication date

queryString = """
SELECT DISTINCT ?IRI ?pYear (COUNT(?pYear) as ?filmNo) WHERE
{
    SELECT DISTINCT ?IRI ?film (MIN(YEAR(?pdate)) as ?pYear) WHERE
    {
        # bind something
        VALUES ?IRI {wd:Q25089 wd:Q3772}
        ?f wdt:P31 wd:Q11424 ;
            wdt:P57 ?IRI ;
            wdt:P577 ?pdate .
        
        # get the label
        ?f sc:name ?film.
    }
    GROUP BY ?IRI ?film
}
GROUP BY ?IRI ?pYear
ORDER BY ?IRI ?pYear
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('IRI', 'http://www.wikidata.org/entity/Q25089'), ('pYear', '1966'), ('filmNo', '1')]
[('IRI', 'http://www.wikidata.org/entity/Q25089'), ('pYear', '1969'), ('filmNo', '1')]
[('IRI', 'http://www.wikidata.org/entity/Q25089'), ('pYear', '1971'), ('filmNo', '1')]
[('IRI', 'http://www.wikidata.org/entity/Q25089'), ('pYear', '1972'), ('filmNo', '1')]
[('IRI', 'http://www.wikidata.org/entity/Q25089'), ('pYear', '1973'), ('filmNo', '1')]
[('IRI', 'http://www.wikidata.org/entity/Q25089'), ('pYear', '1975'), ('filmNo', '1')]
[('IRI', 'http://www.wikidata.org/entity/Q25089'), ('pYear', '1977'), ('filmNo', '1')]
[('IRI', 'http://www.wikidata.org/entity/Q25089'), ('pYear', '1978'), ('filmNo', '1')]
[('IRI', 'http://www.wikidata.org/entity/Q25089'), ('pYear', '1979'), ('filmNo', '1')]
[('IRI', 'http://www.wikidata.org/entity/Q25089'), ('pYear', '1980'), ('filmNo', '1')]
[('IRI', 'http://www.wikidata.org/entity/Q25089'), ('pYear', '1982'), ('filmNo', '1')]
[('IRI', 'http://www.wikidata.org/e

In [19]:
# Query 20

# Finding max no of published films per year for each director. 
# Sorting the result is just for make it easier to review.

# wd:Q25089 => Woody Allen
# wd:Q3772 => Quentin Tarantino
# wdt:P57 => director  (Query 7)
# wdt:P577 => publication date

queryString = """
SELECT ?IRI  (MAX(?filmNo) as ?maxNo)
{
    SELECT DISTINCT ?IRI ?pYear (COUNT(?pYear) as ?filmNo) WHERE
    {
        SELECT DISTINCT ?IRI ?film (MIN(YEAR(?pdate)) as ?pYear) WHERE
        {
            # bind something
            VALUES ?IRI {wd:Q25089 wd:Q3772}
            ?f wdt:P31 wd:Q11424 ;
                wdt:P57 ?IRI ;
                wdt:P577 ?pdate .
            
            # get the label
            ?f sc:name ?film.
        }
        GROUP BY ?IRI ?film
    }
    GROUP BY ?IRI ?pYear
}
GROUP BY ?IRI
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('IRI', 'http://www.wikidata.org/entity/Q25089'), ('maxNo', '2')]
[('IRI', 'http://www.wikidata.org/entity/Q3772'), ('maxNo', '2')]
2


In [17]:
# Query 21

# Now for each director I should compare number of films per year with related maximum.
# I will use the same query as before, but I will add a filter to get only the years with maximum films. 

# I suppose that the the task wants to consider each director separately, so I will use a subquery to get the number of films per year for each one of them.
# In case that the meaning of task was different,
# for example find the year that sum of the films released by all directors are maximum and then find each one contributions, the query would be different.

# wd:Q25089 => Woody Allen
# wd:Q3772 => Quentin Tarantino
# wdt:P57 => director  (Query 7)
# wdt:P577 => publication date

queryString = """
SELECT DISTINCT ?IRI ?Director (GROUP_CONCAT(?pYear; SEPARATOR=", ") as ?Year) (?filmNo as ?noOfFilms) WHERE
{

############################################################################################################
    # get the number of films per year
    # I will use a subquery to get the number of films per year
    # Its output are IRI, pYear and filmNo
    {
        SELECT DISTINCT ?IRI ?pYear (COUNT(?pYear) as ?filmNo) WHERE
        {
            SELECT DISTINCT ?IRI ?f (MIN(YEAR(?pdate)) as ?pYear) WHERE
            {
                # bind something
                VALUES ?IRI {wd:Q25089 wd:Q3772}
                ?f wdt:P31 wd:Q11424 ;
                    wdt:P57 ?IRI ;
                    wdt:P577 ?pdate .
            }
            GROUP BY ?IRI ?f
        }
        GROUP BY ?IRI ?pYear
    }
############################################################################################################
    # get the maximum number of films per year
    # I will use a subquery to get the maximum number of films per year
    # Its output are IRIm and maxNo
    {
        SELECT ?IRIm  (MAX(?filmNo) as ?maxNo)
        {
            SELECT DISTINCT ?IRIm ?pYear (COUNT(?pYear) as ?filmNo) WHERE
            {
                SELECT DISTINCT ?IRIm ?f (MIN(YEAR(?pdate)) as ?pYear) WHERE
                {
                    # bind something
                    VALUES ?IRIm {wd:Q25089 wd:Q3772}
                    ?f wdt:P31 wd:Q11424 ;
                        wdt:P57 ?IRIm ;
                        wdt:P577 ?pdate .
                }
                GROUP BY ?IRIm ?f
            }
            GROUP BY ?IRIm ?pYear
        }
        GROUP BY ?IRIm
    }
############################################################################################################


FILTER(?IRIm = ?IRI && ?filmNo = ?maxNo)

?IRI sc:name ?Director

}
GROUP BY ?IRI  ?Director ?filmNo

LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('IRI', 'http://www.wikidata.org/entity/Q3772'), ('Director', 'Quentin Tarantino'), ('Year', '2003'), ('noOfFilms', '2')]
[('IRI', 'http://www.wikidata.org/entity/Q25089'), ('Director', 'Woody Allen'), ('Year', '1989, 1987, 2001'), ('noOfFilms', '2')]
2


In [21]:
# write your final query

# Query 22

# Now for each director I should compare number of films per year with related maximum.
# I will use the same query as before, but I will add a filter to get only the years with maximum films. 

# I suppose that the the task wants to consider each director separately, so I will use a subquery to get the number of films per year for each one of them.
# In case that the meaning of task was different,
# for example find the year sum of the films released by all directors are maximum and then find each one contributions, the query would be different.

# wd:Q25089 => Woody Allen
# wd:Q3772 => Quentin Tarantino
# wdt:P57 => director  (Query 7)
# wdt:P577 => publication date

queryString = """
SELECT DISTINCT ?IRI ?Director (GROUP_CONCAT(?pYear; SEPARATOR=", ") as ?Year) (?filmNo as ?noOfFilms) WHERE
{

############################################################################################################
    # get the number of films per year
    # I will use a subquery to get the number of films per year
    # Its output are IRI, pYear and filmNo
    {
        SELECT DISTINCT ?IRI ?pYear (COUNT(?pYear) as ?filmNo) WHERE
        {
            SELECT DISTINCT ?IRI ?f (MIN(YEAR(?pdate)) as ?pYear) WHERE
            {
                # bind something
                VALUES ?IRI {wd:Q25089 wd:Q3772}
                ?f wdt:P31 wd:Q11424 ;
                    wdt:P57 ?IRI ;
                    wdt:P577 ?pdate .
            }
            GROUP BY ?IRI ?f
        }
        GROUP BY ?IRI ?pYear
    }
############################################################################################################
    # get the maximum number of films per year
    # I will use a subquery to get the maximum number of films per year
    # Its output are IRIm and maxNo
    {
        SELECT ?IRIm  (MAX(?filmNo) as ?maxNo)
        {
            SELECT DISTINCT ?IRIm ?pYear (COUNT(?pYear) as ?filmNo) WHERE
            {
                SELECT DISTINCT ?IRIm ?f (MIN(YEAR(?pdate)) as ?pYear) WHERE
                {
                    # bind something
                    VALUES ?IRIm {wd:Q25089 wd:Q3772}
                    ?f wdt:P31 wd:Q11424 ;
                        wdt:P57 ?IRIm ;
                        wdt:P577 ?pdate .
                }
                GROUP BY ?IRIm ?f
            }
            GROUP BY ?IRIm ?pYear
        }
        GROUP BY ?IRIm
    }
############################################################################################################


FILTER(?IRIm = ?IRI && ?filmNo = ?maxNo)

?IRI sc:name ?Director

}
GROUP BY ?IRI  ?Director ?filmNo

LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('IRI', 'http://www.wikidata.org/entity/Q3772'), ('Director', 'Quentin Tarantino'), ('Year', '2003'), ('noOfFilms', '2')]
[('IRI', 'http://www.wikidata.org/entity/Q25089'), ('Director', 'Woody Allen'), ('Year', '1989, 1987, 2001'), ('noOfFilms', '2')]
2


## Task 6
Identify the BGP for Academy Awards

In [3]:
# write your queries

# In the result of Query 2, I have it.



Final query for this task

In [8]:
# write your final query

# Query 23


# wd:Q19020 => Academy Awards  (Query 2)

queryString = """
SELECT DISTINCT (wd:Q19020 as ?IRI) ?name
WHERE {
   # bind something
   ?s wdt:P31 wd:Q19020 .
   
   # get the label
   wd:Q19020 sc:name ?name.
}

LIMIT 1
"""

print("Results")
x=run_query(queryString)

Results
[('IRI', 'http://www.wikidata.org/entity/Q19020'), ('name', 'Academy Awards')]
1


## Task 7
Consider films directed by Woody Allen or Quentin Tarantino separately. Return the film which won more Academy Awards for the two directors (the result set must be two elements - one related to Allen and the other related to Tarantino -  and for each element there must be present the director IRI with its label, the film IRI with its label and the number of Academy awards won).

In [18]:
# write your queries

# Query 24

# Ckecking all films from both directors that recieve an Academy Awards.

# Sorting the result is just for make it easier to review.

# wd:Q206124 => 'Midnight in Paris'  (Query 2)
# wdt:P166 => award received (Query 7)
# wd:Q19020 => Academy Awards  (Query 2)
# wd:Q25089 => Woody Allen
# wd:Q3772 => Quentin Tarantino
# wdt:P57 => director 

queryString = """
SELECT DISTINCT ?Director ?fName ?aName
WHERE {
   # bind something
   VALUES ?IRI {wd:Q25089 wd:Q3772}
   
   ?f wdt:P57 ?IRI ;
      wdt:P166 ?a .
      
   ?a wdt:P31 wd:Q19020 .
   
   # get the label
   ?f  sc:name ?fName.
   ?a sc:name ?aName.
   ?IRI sc:name ?Director.
}
ORDER BY ?Director ?fName ?aName
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('Director', 'Quentin Tarantino'), ('fName', 'Django Unchained'), ('aName', 'Academy Award for Best Supporting Actor')]
[('Director', 'Quentin Tarantino'), ('fName', 'Django Unchained'), ('aName', 'Academy Award for Best Writing, Original Screenplay')]
[('Director', 'Quentin Tarantino'), ('fName', 'Inglourious Basterds'), ('aName', 'Academy Award for Best Supporting Actor')]
[('Director', 'Quentin Tarantino'), ('fName', 'Once Upon a Time in Hollywood'), ('aName', 'Academy Award for Best Production Design')]
[('Director', 'Quentin Tarantino'), ('fName', 'Once Upon a Time in Hollywood'), ('aName', 'Academy Award for Best Supporting Actor')]
[('Director', 'Quentin Tarantino'), ('fName', 'Pulp Fiction'), ('aName', 'Academy Award for Best Writing, Original Screenplay')]
[('Director', 'Quentin Tarantino'), ('fName', 'The Hateful Eight'), ('aName', 'Academy Award for Best Original Score')]
[('Director', 'Woody Allen'), ('fName', 'Annie Hall'), ('aName', 'Academy Award for Best Actres

In [44]:

# Query 25

# Counting awards of each film, grouping based on director and film

# Sorting the result is just for make it easier to review.

# wd:Q206124 => 'Midnight in Paris'  (Query 2)
# wdt:P166 => award received (Query 7)
# wd:Q19020 => Academy Awards  (Query 2)
# wd:Q25089 => Woody Allen
# wd:Q3772 => Quentin Tarantino
# wdt:P57 => director 

queryString = """
SELECT DISTINCT ?Director (COUNT(DISTINCT(?aName)) as ?awards)
WHERE {
   # bind something
   VALUES ?IRI {wd:Q25089 wd:Q3772}
   
   ?f wdt:P57 ?IRI ;
      wdt:P166 ?a .
      
   ?a wdt:P31 wd:Q19020 .
   
   # get the label
   ?f  sc:name ?fName.
   ?a sc:name ?aName.
   ?IRI sc:name ?Director.
}
GROUP BY ?Director ?fName
ORDER BY ?Director DESC(?awards)
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('Director', 'Quentin Tarantino'), ('awards', '2')]
[('Director', 'Quentin Tarantino'), ('awards', '1')]
[('Director', 'Woody Allen'), ('awards', '4')]
[('Director', 'Woody Allen'), ('awards', '3')]
[('Director', 'Woody Allen'), ('awards', '1')]
5


In [45]:

# Query 26

# Maximun awards of each director

# Sorting the result is just for make it easier to review.

# wd:Q206124 => 'Midnight in Paris'  (Query 2)
# wdt:P166 => award received (Query 7)
# wd:Q19020 => Academy Awards  (Query 2)
# wd:Q25089 => Woody Allen
# wd:Q3772 => Quentin Tarantino
# wdt:P57 => director 

queryString = """
SELECT DISTINCT ?Director (MAX(?awards) as ?maxAwards) WHERE
{
   SELECT DISTINCT ?Director (COUNT(DISTINCT(?aName)) as ?awards) WHERE
   {
      # bind something
      VALUES ?IRI {wd:Q25089 wd:Q3772}
      
      ?f wdt:P57 ?IRI ;
         wdt:P166 ?a .
         
      ?a wdt:P31 wd:Q19020 .
      
      # get the label
      ?f  sc:name ?fName.
      ?a sc:name ?aName.
      ?IRI sc:name ?Director.
   }
   GROUP BY ?Director ?fName
}
GROUP BY ?Director

LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('Director', 'Woody Allen'), ('maxAwards', '4')]
[('Director', 'Quentin Tarantino'), ('maxAwards', '2')]
2


In [59]:
# Query 27

# Counting awards of each film, grouping based on director and film

# Sorting the result is just for make it easier to review.

# wd:Q206124 => 'Midnight in Paris'  (Query 2)
# wdt:P166 => award received (Query 7)
# wd:Q19020 => Academy Awards  (Query 2)
# wd:Q25089 => Woody Allen
# wd:Q3772 => Quentin Tarantino
# wdt:P57 => director 

queryString = """


SELECT DISTINCT ?IRI ?Director ?filmIRI ?film ?awards WHERE
{

############################################################################################################
# get the number of awards per film for each director
# Output are IRI, f (filmIRI) and awards
   {
   SELECT DISTINCT ?IRI ?f (COUNT(DISTINCT(?a)) as ?awards) WHERE
   {
      # bind something
      VALUES ?IRI {wd:Q25089 wd:Q3772}
      ?f wdt:P57 ?IRI ;
         wdt:P166 ?a .  
      ?a wdt:P31 wd:Q19020 .
   }
   GROUP BY ?IRI ?f
   }
############################################################################################################
# get the maximum of awards per film for each director
# Output are IRIm and maxAwards
   {
   SELECT DISTINCT ?IRIm (MAX(?awards) as ?maxAwards) WHERE
   {
      SELECT DISTINCT ?IRIm (COUNT(DISTINCT(?a)) as ?awards) WHERE
      {
         # bind something
         VALUES ?IRIm {wd:Q25089 wd:Q3772}
         ?f wdt:P57 ?IRIm ;
            wdt:P166 ?a .
         ?a wdt:P31 wd:Q19020 .
      }
      GROUP BY ?IRIm ?f
   }
   GROUP BY ?IRIm
   }
############################################################################################################

   FILTER(?IRI = ?IRIm && ?awards = ?maxAwards)

   # get the label
   ?f  sc:name ?film.
   ?IRI sc:name ?Director.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('IRI', 'http://www.wikidata.org/entity/Q3772'), ('Director', 'Quentin Tarantino'), ('film', 'Django Unchained'), ('awards', '2')]
[('IRI', 'http://www.wikidata.org/entity/Q3772'), ('Director', 'Quentin Tarantino'), ('film', 'Once Upon a Time in Hollywood'), ('awards', '2')]
[('IRI', 'http://www.wikidata.org/entity/Q25089'), ('Director', 'Woody Allen'), ('film', 'Annie Hall'), ('awards', '4')]
3


Final query for this task

In [60]:
# write your final query

# Query 28

# What is done is very similar to Task 5. 
# Note that filmIRIs and filmTitles are in same order in concatenated output result.

# wdt:P166 => award received (Query 7)
# wd:Q19020 => Academy Awards  (Query 2)
# wd:Q25089 => Woody Allen
# wd:Q3772 => Quentin Tarantino
# wdt:P57 => director 

queryString = """

SELECT DISTINCT ?IRI ?Director (GROUP_CONCAT(?f; SEPARATOR=", ") as ?filmIRI) (GROUP_CONCAT(?film; SEPARATOR=", ") as ?filmtitle) ?awards WHERE
{

############################################################################################################
# get the number of awards per film for each director
# Output are IRI, f (filmIRI) and awards
   {
   SELECT DISTINCT ?IRI ?f (COUNT(DISTINCT(?a)) as ?awards) WHERE
   {
      # bind something
      VALUES ?IRI {wd:Q25089 wd:Q3772}
      ?f wdt:P57 ?IRI ;
         wdt:P166 ?a .  
      ?a wdt:P31 wd:Q19020 .
   }
   GROUP BY ?IRI ?f
   }
############################################################################################################
# get the maximum of awards per film for each director
# Output are IRIm and maxAwards
   {
   SELECT DISTINCT ?IRIm (MAX(?awards) as ?maxAwards) WHERE
   {
      SELECT DISTINCT ?IRIm (COUNT(DISTINCT(?a)) as ?awards) WHERE
      {
         # bind something
         VALUES ?IRIm {wd:Q25089 wd:Q3772}
         ?f wdt:P57 ?IRIm ;
            wdt:P166 ?a .
         ?a wdt:P31 wd:Q19020 .
      }
      GROUP BY ?IRIm ?f
   }
   GROUP BY ?IRIm
   }
############################################################################################################

   FILTER(?IRI = ?IRIm && ?awards = ?maxAwards)

   # get the label
   ?f  sc:name ?film.
   ?IRI sc:name ?Director.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('IRI', 'http://www.wikidata.org/entity/Q25089'), ('Director', 'Woody Allen'), ('filmIRI', 'http://www.wikidata.org/entity/Q233464'), ('filmtitle', 'Annie Hall'), ('awards', '4')]
[('IRI', 'http://www.wikidata.org/entity/Q3772'), ('Director', 'Quentin Tarantino'), ('filmIRI', 'http://www.wikidata.org/entity/Q571032, http://www.wikidata.org/entity/Q47300912'), ('filmtitle', 'Django Unchained, Once Upon a Time in Hollywood'), ('awards', '2')]
2


## Task 8
Consider the set of films directed by Allen or Tarantino. Return for each Academy Award the total number of nomination of these films (the result set must be a list of triples Academy Award IRI, label and #nomination).

In [65]:
# write your queries

# Query 29

# Finding list of all films from directors nominated for any kind of Academy Awards to have a clue.
# Sorting the result is just for make it easier to review.

# wd:Q19020 => Academy Awards
# wdt:P1411 => nominated for  (Query 7)

queryString = """
SELECT DISTINCT ?Director ?fName ?aName
WHERE {
   # bind something
   VALUES ?IRI {wd:Q25089 wd:Q3772}
   ?f wdt:P57 ?IRI ;
      wdt:P1411 ?a .
   ?a wdt:P31 wd:Q19020 .
   
   # get the label
   ?f  sc:name ?fName.
   ?a sc:name ?aName.
   ?IRI sc:name ?Director.
}
ORDER BY ?Director ?fName ?aName
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('Director', 'Quentin Tarantino'), ('fName', 'Django Unchained'), ('aName', 'Academy Award for Best Cinematography')]
[('Director', 'Quentin Tarantino'), ('fName', 'Django Unchained'), ('aName', 'Academy Award for Best Picture')]
[('Director', 'Quentin Tarantino'), ('fName', 'Django Unchained'), ('aName', 'Academy Award for Best Sound Editing')]
[('Director', 'Quentin Tarantino'), ('fName', 'Django Unchained'), ('aName', 'Academy Award for Best Supporting Actor')]
[('Director', 'Quentin Tarantino'), ('fName', 'Django Unchained'), ('aName', 'Academy Award for Best Writing, Original Screenplay')]
[('Director', 'Quentin Tarantino'), ('fName', 'Inglourious Basterds'), ('aName', 'Academy Award for Best Cinematography')]
[('Director', 'Quentin Tarantino'), ('fName', 'Inglourious Basterds'), ('aName', 'Academy Award for Best Director')]
[('Director', 'Quentin Tarantino'), ('fName', 'Inglourious Basterds'), ('aName', 'Academy Award for Best Film Editing')]
[('Director', 'Quentin Taran

In [66]:
# Query 30

# Counting no of nominated films for each Academic Awards.
# Sorting the result is just for make it easier to review.

# wd:Q19020 => Academy Awards
# wdt:P1411 => nominated for  (Query 7)

queryString = """
SELECT DISTINCT  ?aName (COUNT(?f) as ?noOfFilm)
WHERE {
   # bind something
   VALUES ?IRI {wd:Q25089 wd:Q3772}
   ?f wdt:P57 ?IRI ;
      wdt:P1411 ?a .
   ?a wdt:P31 wd:Q19020 .
   
   # get the label
   ?a sc:name ?aName.
}
GROUP BY ?aName
ORDER BY ?aName
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('aName', 'Academy Award for Best Actor'), ('noOfFilm', '3')]
[('aName', 'Academy Award for Best Actress'), ('noOfFilm', '3')]
[('aName', 'Academy Award for Best Cinematography'), ('noOfFilm', '4')]
[('aName', 'Academy Award for Best Costume Design'), ('noOfFilm', '2')]
[('aName', 'Academy Award for Best Director'), ('noOfFilm', '9')]
[('aName', 'Academy Award for Best Film Editing'), ('noOfFilm', '3')]
[('aName', 'Academy Award for Best Original Score'), ('noOfFilm', '1')]
[('aName', 'Academy Award for Best Picture'), ('noOfFilm', '6')]
[('aName', 'Academy Award for Best Production Design'), ('noOfFilm', '5')]
[('aName', 'Academy Award for Best Sound'), ('noOfFilm', '1')]
[('aName', 'Academy Award for Best Sound Editing'), ('noOfFilm', '2')]
[('aName', 'Academy Award for Best Supporting Actor'), ('noOfFilm', '7')]
[('aName', 'Academy Award for Best Supporting Actress'), ('noOfFilm', '11')]
[('aName', 'Academy Award for Best Writing, Original Screenplay'), ('noOfFilm', '19')]


Final query for this task

In [70]:
# write your final query

# Query 31

# Sorting the result is just for make it easier to review.

# wd:Q19020 => Academy Awards
# wdt:P1411 => nominated for  (Query 7)

queryString = """
SELECT DISTINCT (?a as ?awardIRI) ?award (COUNT(?f) as ?noOfNominates)
WHERE {
   # bind something
   VALUES ?IRI {wd:Q25089 wd:Q3772}
   ?f wdt:P57 ?IRI ;
      wdt:P1411 ?a .
   ?a wdt:P31 wd:Q19020 .
   
   # get the label
   ?a sc:name ?award.
}
GROUP BY ?a ?award
ORDER BY ?award
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('awardIRI', 'http://www.wikidata.org/entity/Q103916'), ('award', 'Academy Award for Best Actor'), ('noOfNominates', '3')]
[('awardIRI', 'http://www.wikidata.org/entity/Q103618'), ('award', 'Academy Award for Best Actress'), ('noOfNominates', '3')]
[('awardIRI', 'http://www.wikidata.org/entity/Q131520'), ('award', 'Academy Award for Best Cinematography'), ('noOfNominates', '4')]
[('awardIRI', 'http://www.wikidata.org/entity/Q277536'), ('award', 'Academy Award for Best Costume Design'), ('noOfNominates', '2')]
[('awardIRI', 'http://www.wikidata.org/entity/Q103360'), ('award', 'Academy Award for Best Director'), ('noOfNominates', '9')]
[('awardIRI', 'http://www.wikidata.org/entity/Q281939'), ('award', 'Academy Award for Best Film Editing'), ('noOfNominates', '3')]
[('awardIRI', 'http://www.wikidata.org/entity/Q488651'), ('award', 'Academy Award for Best Original Score'), ('noOfNominates', '1')]
[('awardIRI', 'http://www.wikidata.org/entity/Q102427'), ('award', 'Academy Award for